In [2]:
import pandas as pd
from generate_process_errors import generate_error_df, splice_sentences, save_iterative_errors

api_key = '' # put in your api key
api_base = "" # put in the azure base site
mimic_path = '/Users/raovish6/Documents/MRG_entailment/mimic_train_subset_reports_new.csv' # path to full mimic train, preprocessed by lab
mimic_df = pd.read_csv(mimic_path)
testing_df = mimic_df[:10]

In [2]:
# Example 1: create an error df based on a custom system prompt (assuming the prompt specifies a new line between the generated error report and sentence labeling)
system_prompt = "You will be given a radiology report of a chest X-ray. Your task is to change some of the statements in the report so that the report is still clinically plausible but has a different meaning than the previous report. Keep track of the sentence indexes corresponding to the sentences you change in a report.\nFor a given report, return a new report with one or more changed sentences according to the above paragraph, a new line, and then a Python dictionary in the following format: {error sentence index : label, explanation, original sentence index]}. Make sure this format is followed exactly, including the spacing. The label is determined by the following:\n0: unchanged sentence\n1: changed sentence\n‘explanation’ is determined by the following:\nWhen the label is 1: 'explanation' should contain one statement about the change made in the sentence. The length of the statement should not exceed 15 words. When the 'label' is 0: 'explanation' should contain 'not applicable'. The error sentence indices should match the total number of sentences of the error report, after any additions, and include an entry for every sentence in the error report. For example, an error report of three sentences should have a dictionary of {0: [label, explanation, original sentence index], 1: [label, explanation, original sentence index], 2: [label, explanation, original sentence index]}. The original sentence index refers to the sentence  index of the original report that was changed. If the error is not based on a sentence from the original report (e.g. an addition) or the sentence is correct, leave the original sentence index blank."

error_df = generate_error_df(testing_df, api_key, api_base, system_prompt, False)
error_df.head()

,Study ID,Subject ID,Original Report,Error Report,Sentence Labelings
0,50414267,10000032,"There is no focal consolidation, pleural effus...","Focal consolidation is evident, with no pleura...","{0: [1, 'Changed no consolidation to evident c..."


In [4]:
# splice error_df into sentence-by-sentence labeling:
spliced_df = splice_sentences(error_df)
spliced_df.head()

,Study ID,Subject ID,Original Sentence,Error Sentence,Sequence,Label,Error Class
0,50414267,10000032,"There is no focal consolidation, pleural effus...","Focal consolidation is evident, with no pleura...",0,1,Changed no consolidation to evident consolidation
1,50414267,10000032,,Bilateral nodular opacities are noted that po...,1,0,not applicable
2,50414267,10000032,,The cardiomediastinal silhouette appears normal,2,0,not applicable
3,50414267,10000032,"Clips project over the left lung, potentiall...","Clips project over the right lung, potentiall...",3,1,Changed left lung to right lung
4,50414267,10000032,,The imaged upper abdomen is unremarkable,4,0,not applicable


In [3]:
import os

# Example 2: given a dictionary of errors to try, try each of them individually and save the outputs within a root directory of choice

error_dict = {
    0: "add repetitions of sentences (in this case, mark the second instance of the sentence as the error, not the first). Have one repetition per report.",
    1: "introduce sentences that could be part of a radiology report regarding the presence of one or more of devices such as these: pacemaker, central venous line, NG tube, ET tube. Use your medical knowledge to frame sentences that sound clinically plausible.",
    2: "use your medical knowledge to change the position of a medical instrument that is clinically plausible (e.g., ET tube is 5 cm from the carina can be changed to ET tube is 1 cm from the carina).",
    3: "add incorrect severity: change the severity of a finding in the report in a manner that makes clinical sense (e.g., change 'mild' to 'moderate'; always modifying a sentence)."
}

root_path = '/Users/raovish6/Downloads/iterative_tests/retrial-2/'
if not os.path.exists(root_path):
    os.mkdir(root_path)

# if it does not run through all the dictionary indices on the first run through, re-send the un-run indices as last argument (happens cuz api gets stuck sometimes)
save_iterative_errors(error_dict, testing_df, api_key, api_base, root_path, [0])


0
0
1
2
3
4
5
6
7
8
9
1
0
1
2
3
4
5
6
7
8
9
3
0
1
2
3
4
5
6
7
8
9
